Train generator model (finetune gpt2) on google colab.  Run this as a TPU notebook.

This is a lightweight wrapper around `train.py` that calls it from a notebook.

You should run this after you've prepared a training corpus (`prep_generator_training_dataset.ipynb`).

As in other notebooks, you should have a google drive folder with my fork of gpt2 (included as `gpt-2/`).  You should have downloaded the base 1558M model using `download_model.py`.

In [ ]:
# mount drive, setup
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/gpt-2"
!pip3 install -r "requirements.txt"

Train.

You must have ~6gb left on google drive for the checkpoint.  You should expect to get ~350 steps of training done before colab times out.  You can continue from the checkpoint repeatedly after that, possibly after waiting a little while to get TPU usage back for your account.

Alas, I have not added anything to save optimizer state in the checkpoints, so the adam gradient accumulators will get reset every ~350 steps.

Heuristics for how many steps to run:

- you will see something like 'dataset has XX tokens'.
  - divide XX by (1024 * 8).  this is how long an 'epoch' is, in steps
  - i used to do roughly 1 epoch at learning rate ~1.4e-5.
  - now i do roughly 2 to 2.5 epochs at learning rate ~6.5e-6, which is used below.
  - so, i recommend doing roughly 2 to 2.5 epochs
- alternately, look at the loss
  - early on, avg loss should be in the 2.9-3.0 range.
  - a "good" finetuned model should get below this to maybe ~2.5 or lower
  - too low (eg <2) is bad
- also, review the samples generated, and consider stopping if you like them at a given point

In [ ]:
PATH_TO_YOUR_DATA = ""  # a directory you put your data in (text files or .npz)
YOUR_MODEL_NAME = ""  # whatever you want to call your model

In [ ]:

!PYTHONPATH=/tensorflow-1.15.2/python3.6:/env/python:src/ python3 -u train.py \
        --model_name 1558M \
        --run_name {YOUR_MODEL_NAME} \
        --dataset {PATH_TO_YOUR_DATA} \
        --batch_size 8 \
        --learning_rate 0.0000065 \
        --eot_workaround \
        --rob_sampler \
        --save_every 20 \
        --sample_every 100 \
        --sample_num 8 \
        --sample_length 192 \
        --save_time 30000  \
        --max_to_keep 1 \
        --init_tpu

after finishing training, create a directory under `models` with the model, and metadata needed to load the it and the corresponding encoder.

In [ ]:
!mkdir models/{YOUR_MODEL_NAME}

!cp models/1558M/hparams.json models/{YOUR_MODEL_NAME}
!cp models/1558M/vocab.bpe models/{YOUR_MODEL_NAME}
!cp models/1558M/encoder.json models/{YOUR_MODEL_NAME}